# HANDLING OUTLIERS

## TUJUAN :  mendeteksi dan mengangi data ekstrem yang bisa merusak model
kapan dipakai : setelah load data , sebelum training model

kenapa penting?
- outlier bisa bikin modmels prediksi melenceng jauh
- contoh : dat agaji [5jt,6jt,64t,500jt] -> 500jt adalah outlier
- model jadi fokus ke outlier, buka pola normal

metode yang akan di pelajari :
1. IQR method ( interquartikle range ) - paling umum
2. Z-Score method - untuk data distribusi normal
3. visualisasi dengan boxplot
4. strartegi handling : remove,cap,transform

In [5]:
import numpy as np
import pandas as pd


buat data dummy dengan outlier

In [6]:
print("\n membuat data dummy dengan ourlier \n")

np.random.seed(42)  #mengunci" hasil pengacakan agar urutan angka acak yang dihasilkan selalu sama setiap kali kode dijalankan.

data = {
    'Nama' : ['Emp_' + str(i) for i in range(1,21)],
    'Umur'  : [25,28,30,27,29,26,31,24,28,30,
               27,29,26,25,28,30,27,65,29,26], # 65 = outlier
    'Gaji'  : [500000,550000,600000,520000,580000,
               530000,620000,480000,560000,610000,
               540000,590000,510000,5000000,570000,
               600000,550000,2500000,530000,520000], # 250 jt outlier
    'Tahun_kerja'   : [2,3,5,3,4,2,6,1,4,5,
                       3,4,2,2,4,5,3,15,4,2] # 15 out;ier
}

df = pd.DataFrame(data)

print("datra awal (perhtaikan baris 18 - ada outlier )")
print(df)
print(f"\n jumlah data : {len(df)} baris")


 membuat data dummy dengan ourlier 

datra awal (perhtaikan baris 18 - ada outlier )
      Nama  Umur     Gaji  Tahun_kerja
0    Emp_1    25   500000            2
1    Emp_2    28   550000            3
2    Emp_3    30   600000            5
3    Emp_4    27   520000            3
4    Emp_5    29   580000            4
5    Emp_6    26   530000            2
6    Emp_7    31   620000            6
7    Emp_8    24   480000            1
8    Emp_9    28   560000            4
9   Emp_10    30   610000            5
10  Emp_11    27   540000            3
11  Emp_12    29   590000            4
12  Emp_13    26   510000            2
13  Emp_14    25  5000000            2
14  Emp_15    28   570000            4
15  Emp_16    30   600000            5
16  Emp_17    27   550000            3
17  Emp_18    65  2500000           15
18  Emp_19    29   530000            4
19  Emp_20    26   520000            2

 jumlah data : 20 baris


method 1 iqr  ( interquartile range ) - paling populer

konsep
- Q1 (kuartil 1 ) = persentil ke -25
- Q3 ( kuartil 3 ) = persenti = 75
- IQR = Q3 - Q1
- ourlier jika:
* Data < Q1 - 1.5*IQR ( outlier bawah )
* Data > Q3 + 1.5 * IQR  ( outlier atas )

In [17]:
def detect_outlier_iqr(df, column):
    """
    fungsi untuk deteksi outlier dengan metode IQR

    args :
    df : dataframe
    column : sNama kolom yang mau di cek

    return:
    dictionary berisi info outlier
    """

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # deteksi outlier
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return {
        'Q1'    : Q1,
        'Q3' : Q3,
        'IQR' : IQR,
        'lower_bound' : lower_bound,
        'upper_bound' : upper_bound,
        'outliers' : outliers,
        'jumlah_outlier' : len(outliers)
    }

# cek outlier di kolom gaji

print(" deteksi outlier di kolom 'Gaji' : ")
result_gaji = detect_outlier_iqr(df, 'Gaji')

print(f"Q1 ( 25%) : RP {result_gaji['Q1']:,.0f}")
print(f"Q3 ( 75%) : RP {result_gaji['Q3']:,.0f}")
print(f"IQR : Rp {result_gaji['IQR']:,.0f}")
print(f"Batas bawah : Rp {result_gaji['lower_bound']:,.0f}")
print(f"Batas Atas : Rp {result_gaji['upper_bound']:,.0f}")
print(f"\n Jumlah outlier : Rp {result_gaji['jumlah_outlier']:,.0f}")

if result_gaji['jumlah_outlier'] > 0:
    print("\nDARA OUTLIER : ")
    print(result_gaji['outliers'][['Nama', 'Gaji']])

# cek outlier di kolom umur
print("\n Deteksi umur di kolom umur : ")
result_umur = detect_outlier_iqr(df, 'Umur')
print(f"batas normal : {result_umur['lower_bound']:,.1f} - {result_umur['upper_bound']:,.1f}")
print(f"Jumlah Outlier: {result_umur['jumlah_outlier']}")

if result_umur['jumlah_outlier'] > 0:
    print("\nData Outlier : ")
    print(result_umur['outliers'][['Nama', 'Umur']])



 deteksi outlier di kolom 'Gaji' : 
Q1 ( 25%) : RP 527,500
Q3 ( 75%) : RP 600,000
IQR : Rp 72,500
Batas bawah : Rp 418,750
Batas Atas : Rp 708,750

 Jumlah outlier : Rp 2

DARA OUTLIER : 
      Nama     Gaji
13  Emp_14  5000000
17  Emp_18  2500000

 Deteksi umur di kolom umur : 
batas normal : 21.1 - 34.1
Jumlah Outlier: 1

Data Outlier : 
      Nama  Umur
17  Emp_18    65


METODE  2 Z - SCORE METHOD

KONSEP :
- Z-SCORE = (nilai - mean ) /std_dev
- mengukur seberapa jauh data dari rata rata
- outlier jika \ Z-score | > 3 ( sangat jauh dari rata rata )

cocok untuk data yang distribusinya normal ) bell curve)

In [11]:
def detect_outlier_zscore(df, column, threshold=3):
    """
    deteksi outlier dengan z-score

    args :
    df :DataFrame
    columm : Nama Kolom
    threshold : batas z-score default = 3

    return :  data frame dengan kolom z-score
    :param df:
    :param column:
    :param threshold:
    :return:
    """

    # hitung z-score
    mean = df[column].mean()
    std = df[column].std()

    df[f'{column}_Zscore'] = (df[column] - mean) / std

    # tandai outlier

    df[f'{column}_IsOutlier'] = np.abs(df[f'{column}_Zscore']) > threshold

    outliers = df[df[f'{column}_IsOutlier'] == True]

    print(f"mean : {mean:2f}")
    print(f"std dev : {std:2f}")
    print(f"threshold : {threshold}")
    return outliers


print(" deteksi dengan Z_score pada 'gaji' : ")
outlier_z = detect_outlier_zscore(df, 'Gaji')
if len(outlier_z) > 0:
    print(outlier_z[['Nama', 'Gaji', 'Gaji_Zscore']])


 deteksi dengan Z_score pada 'gaji' : 
mean : 873000.000000
std dev : 1064909.682358
threshold : 3
      Nama     Gaji  Gaji_Zscore
13  Emp_14  5000000     3.875446


## STARATEGI HANDLING OUTLIER

APA YANG DILAKUKKAN OUTLIER

STRATEGI = REMOVE
KAPAN "OUTLIER ADALAH DATA ERROR / SALAH INPUT

In [28]:
# HAPUS OUTLIER BERDASARKAN IQR

df_cleaned = df.copy()
Q1 = df_cleaned['Gaji'].quantile(0.25)
Q3 = df_cleaned['Gaji'].quantile(0.75)
IQR = Q3 - Q1
lower =Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_cleaned = df_cleaned[(df_cleaned['Gaji'] >= lower) & (df_cleaned['Gaji'] <= upper)]

print(f"    Data sebelum : {len(df)} Baris")
print(f"    Data sesidah : {len(df_cleaned)} Baris")
print(f"    Dihapus : {len(df) - len(df_cleaned)} Baris")

#STRATEGI KE 2  : CAPPING ( batasi nilai )
print("\n Strategi CAPPING ( Winsorization)")
print("     Kapan : Outlier adalah data valid tapi ekstream")
print()

df_capped = df.copy()

# cap ke abtas atas dan bawah
df_capped['Gaji_Cappped'] = df_capped['Gaji'].clip(lower=lower,upper=upper)

print(f"    Gaji original Max : Rp {df['Gaji'].max():,.0f}")
print(f"    Gaji capped Max : Rp {df_capped['Gaji_Cappped'].max():,.0f}")
print(f"    Upper Bound : Upper bound: Rp {upper:,.0f}")

# Strategi 3 transformation ( transformasi )
print(f"\n  STRATEGI : TRANSFORMASI")
print("     Kapan : outlier banyak, distribusi Data skewed")
print()

df_transformed = df.copy()

# log transformation ( untuk data positif )
df_transformed['Gaji_log'] = np.log1p(df_transformed['Gaji'])   #log1p = log(1+x)

print("     Original gaji range : ", df['Gaji'].min(), "-", df['Gaji'].max())
print("     Log Transformed range : ", df_transformed['Gaji_log'].min().round(2), "-", df_transformed["Gaji_log"].max().round(2))


#==================================================
# FINGSI HELPER: CEK SEMUA KOLOM NUMERIK
#===================================================
print("\n" + "=" * 60)
print("BONUS , FUNGSI UNTUK CEK SEMUA KOLOM SEKALIGUS")
print("=" * 60)
print()

def check_all_outliers(df, method='iqr'):
    """
    cek outlier di semua kolom numerik

    args :
    df : DataFrame
    method : iqr atau zscore
    """

    numeric_col = df.select_dtypes(include=[np.number]).columns

    print(f"\n Mengecek {len(numeric_col)} kolom numerik dengan methode {method.upper()} : ")
    print("-" * 60)

    for col in numeric_col:
        if method == 'iqr':
            result = detect_outlier_iqr(df, col)
            print(f"]n {col}")
            print(f" Range normal : {result['lower_bound']:,.0f} - {result['upper_bound']:,.0f}")
            print(f" Outliers : {result['jumlah_outlier']}")

        elif method == 'zscore':
            mean = df[col].mean()
            std = df[col].std()
            z_scores = np.abs((df[col]- mean)/ std)
            outliers = (z_scores > 3).sum()
            print(f"\n{col}: {outliers} outliers ( Z_score > 3")

check_all_outliers(df, method='iqr')


    Data sebelum : 20 Baris
    Data sesidah : 18 Baris
    Dihapus : 2 Baris

 Strategi CAPPING ( Winsorization)
     Kapan : Outlier adalah data valid tapi ekstream

    Gaji original Max : Rp 5,000,000
    Gaji capped Max : Rp 708,750
    Upper Bound : Upper bound: Rp 708,750

  STRATEGI : TRANSFORMASI
     Kapan : outlier banyak, distribusi Data skewed

     Original gaji range :  480000 - 5000000
     Log Transformed range :  13.08 - 15.42

BONUS , FUNGSI UNTUK CEK SEMUA KOLOM SEKALIGUS


 Mengecek 4 kolom numerik dengan methode IQR : 
------------------------------------------------------------
]n Umur
 Range normal : 21 - 34
 Outliers : 1
]n Gaji
 Range normal : 418,750 - 708,750
 Outliers : 2
]n Tahun_kerja
 Range normal : -1 - 8
 Outliers : 1
]n Gaji_Zscore
 Range normal : -0 - -0
 Outliers : 2


# KESIMPULAN

*KAPAN PAKAI IQR :*
- Data tidak terdistribusi normal
-  Ingin method robust ( tidak terpengaruh outlier )
- paling umum di industri

*KAPAN Z_SCORE*
- data terdistribusi normal
- ingin detekesi outlier ekstrem (>3 std dari aman )

*STRATEGI HANDLING:
- REMOVE -> jika yakin data error ( typo, sensor rusak )
- capping -> jika data valid tapi ekstrem (CEO SALARY )
- Transform -> jika banyak outlier & distribusi skewed

*JANGAN :*
- Hapus outlier tanpa investigasi terlebih dahulu
- Hapus terlalu banyak data ( >5% dari total )
- Abaikan domain knowlede (GAJI CEO )

*TIPS*
- selalu visualisasika dengan boxplot
- disukusi dengan domain expert
- dokumentasikan alasan handling
